In [1]:
import pandas as pd
import pysolr
import os

In [2]:
# establish connection with the core
core_path = "http://localhost:8983/solr/stock_project_core"
solr = pysolr.Solr(core_path, always_commit=True)

# Remove all indexes from the core

In [3]:
# remove all index
solr.delete(q='*:*')

'<?xml version="1.0" encoding="UTF-8"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">71</int>\n</lst>\n</response>\n'

# Index data from csv

In [4]:
# load csv data (labelling version)
csv_path = "IR_New_Testing_Data - Tesing_Data.csv"

csv_data = pd.read_csv(csv_path, dtype={'tweet_id': 'int64'}, float_precision='round_trip')
csv_data['tweet_id'] = csv_data['tweet_id'].astype(str)
csv_data.head(5)

,ticker_symbol,post_date,tweet_id,writer,body,like_num,comment_num,retweet_num,tweet_activity,url_cnt,emoticons,emojis,emo_cnt,word_count,clean_text,subjectivity,sentiment,moderator,clean_text_no_stem_user
0,AAPL,2023-01-01 22:23:31+00:00,1609676719925137408,David_Tracey,Things are going to get very interesting but w...,27,1,2,30,2,[],{},0,11,thing go get interest q1 forecast updat aapl r...,0,0.0,NaN,Things going get interesting My Q1 forecast So...
1,AAPL,2023-01-01 21:04:45+00:00,1609656899674284032,ASTS_Investors,Happy New Year SpaceMob! \n\nLet's hope 2023 w...,64,0,3,67,1,[],"{'🚀': 1, '🤠': 1}",2,21,happi new year spacemob let hope 2023 year blu...,1,1.0,NaN,Happy New Year SpaceMob Lets hope 2023 year Bl...
2,AAPL,2023-01-01 20:44:51+00:00,1609651892447023104,BigCheds,$AAPL Apple weekly chart - hammer candle with ...,104,9,5,118,1,[],{},0,10,aapl appl weekli chart hammer candl spring jun...,0,0.0,NaN,AAPL Apple weekly chart hammer candle spring J...
3,AAPL,2023-01-01 19:45:30+00:00,1609636954454343680,ThetaWarrior,$AAPL Yearly -26.8% https://t.co/SewErQmOPE,80,10,10,100,1,[],{},0,4,aapl yearli 268,0,0.0,NaN,AAPL Yearly 268
4,AAPL,2023-01-01 19:00:00+00:00,1609625505287790592,TrendSpider,Looking for shorts into the new year? 📉\n\nThi...,28,4,4,36,1,[],{'📉': 1},1,23,look short new year weekli triangl breakdown s...,1,1.0,NaN,Looking shorts new year This weekly triangle b...


In [5]:
# extract csv data to be indexed
extracted_data = []
for index, row in csv_data.iterrows():
    extracted_data.append({"id": row["tweet_id"],
                           "company": row["ticker_symbol"],
                           "post_date": row["post_date"],
                            "author": row["writer"], 
                            "raw_text": row["body"], 
                            "like_num": row["like_num"],
                            "comment_num": row["comment_num"], 
                            "retweet_num": row["retweet_num"], 
                            "tweet_activity": row["tweet_activity"],
                            "clean_text": row["clean_text_no_stem_user"],
                            "subjectivity": row["subjectivity"], 
                            "sentiment": row["sentiment"]})

In [6]:
# define company names
ticker_symbol_to_name_mapping = {"AAPL": "apple",
                                "AMZN": "amazon",
                                "GOOGL": "google",
                                "MSFT": "Microsoft",
                                "TSLA": "Tesla"}

for i in range(len(extracted_data)):
    extracted_data[i]["company"] = ticker_symbol_to_name_mapping[extracted_data[i]["company"]]

In [7]:
# inspect data before indexing
extracted_data

[{'id': '1609676719925137408',
  'company': 'apple',
  'post_date': '2023-01-01 22:23:31+00:00',
  'author': 'David_Tracey',
  'raw_text': 'Things are going to get very interesting but when...?\nMy Q1 forecast below...\n\nSome updates here on $AAPL too as requested.\n\nhttps://t.co/dU1FcS22JP https://t.co/x8RhRPXHhD',
  'like_num': 27,
  'comment_num': 1,
  'retweet_num': 2,
  'tweet_activity': 30,
  'clean_text': 'Things going get interesting My Q1 forecast Some updates AAPL requested',
  'subjectivity': 0,
  'sentiment': 0.0},
 {'id': '1609656899674284032',
  'company': 'apple',
  'post_date': '2023-01-01 21:04:45+00:00',
  'author': 'ASTS_Investors',
  'raw_text': "Happy New Year SpaceMob! \n\nLet's hope 2023 will be the year of #BlueWalker3 and $ASTS 🚀🤠\n\n#Space5G #5G $GSAT $T $AMT $TMUS $AAPL $IRDM #SpaceX $VOD https://t.co/cefLRpuvEl",
  'like_num': 64,
  'comment_num': 0,
  'retweet_num': 3,
  'tweet_activity': 67,
  'clean_text': 'Happy New Year SpaceMob Lets hope 2023 year Bl

In [8]:
# index data
print(solr.add(extracted_data))

{
  "responseHeader":{
    "status":0,
    "QTime":1812}}



# Query guide

Note that like_num, comment_num, retweet_num, tweet_activity, subjectivity, sentiment are not indexed and hence can't be queried. However, their values can be returned when relevant queries are made using other query fields such as id.

## Single field query, query field = text body

When querying the text body, query with clean_text, display raw_text

In [17]:
def display_results(search_results, query_field):
    # show spell correction only if query_field == "clean_text"
    if query_field == "clean_text":
        if len(search_results) == 0 and search_results.spellcheck['suggestions'][1]['numFound'] > 0:
            print("It seems the query you are searching if not present in the database. Here are some alternatives suggested queries: ")
            print(search_results.spellcheck['suggestions'][1]['suggestion'])
        else:
            for result in search_results:
                print("id: ", result["id"])
                print("company: ", result["company"])
                print("post_date: ", result["post_date"])
                print("author: ", result["author"])
                print("raw_text: ", result["raw_text"])
                print("like_num: ", result["like_num"])
                print("comment_num: ", result["comment_num"])
                print("retweet_num: ", result["retweet_num"])
                print("tweet_activity: ", result["tweet_activity"])
                print("subjectivity: ", result["subjectivity"])
                print("sentiment: ", result["sentiment"])
                print("====================================")
                print()
    else:
        if len(search_results) == 0:
            print("It seems the query you are searching if not present in the database")
        else:
            for result in search_results:
                print("id: ", result["id"])
                print("company: ", result["company"])
                print("post_date: ", result["post_date"])
                print("author: ", result["author"])
                print("raw_text: ", result["raw_text"])
                print("like_num: ", result["like_num"])
                print("comment_num: ", result["comment_num"])
                print("retweet_num: ", result["retweet_num"])
                print("tweet_activity: ", result["tweet_activity"])
                print("subjectivity: ", result["subjectivity"])
                print("sentiment: ", result["sentiment"])
                print("====================================")
                print() 

In [18]:
# query on 1 field
query_field = "clean_text" # define the field to search under
query = "economically" # define the search term
fields_to_return = "id, company, post_date, author, raw_text, like_num, comment_num, retweet_num, tweet_activity, subjectivity, sentiment" # define the fields to return results
top_n_results = 3 # define the number of top results to return

query_string = "%s: %s" % (query_field, query)

search_results = solr.search(query_string, 
                             **{'fl': fields_to_return, 
                                'rows': top_n_results})

display_results(search_results, query_field)

id:  1610730772305936384
company:  ['Microsoft']
post_date:  2023-01-04T20:11:56Z
author:  ['NestBetter']
raw_text:  ['@LiebermanAustin C) there is no threat but $MSFT is about to force their hand and worsen the economics of search']
like_num:  [19]
comment_num:  [4]
retweet_num:  [0]
tweet_activity:  [23]
subjectivity:  [1]
sentiment:  [0]

id:  1614029250730000384
company:  ['Tesla']
post_date:  2023-01-13T22:38:55Z
author:  ['Teslaconomics']
raw_text:  ['I believe 2M vehicles sold in 2023 is in the bag, barring any crazy economic event. $TSLA']
like_num:  [493]
comment_num:  [36]
retweet_num:  [33]
tweet_activity:  [562]
subjectivity:  [1]
sentiment:  [1]

id:  1624473691408203776
company:  ['apple']
post_date:  2023-02-11T18:21:23Z
author:  ['altcap']
raw_text:  ['@tanayj Non consensus view.  Chat GPTs generally squeeze search economics which rolls downhill - less economics to google / msft search means less rev share to $aapl.  Further - Siri upgrades its own chat GPT capabilities

## Single field query, query field = post date

In [19]:
# query on 1 field
query_field = "post_date" # define the field to search under
query = '[2023-01-01T23\:59\:59Z TO 2023-03-01T23\:59\:59Z]' # define the search term
fields_to_return = "id, company, post_date, author, raw_text, like_num, comment_num, retweet_num, tweet_activity, subjectivity, sentiment" # define the fields to return results
top_n_results = 3 # define the number of top results to return

query_string = "%s: %s" % (query_field, query)

search_results = solr.search(query_string, 
                             **{'fl': fields_to_return, 
                                'rows': top_n_results})

display_results(search_results, query_field)

id:  1610040852876857344
company:  ['apple']
post_date:  2023-01-02T22:30:27Z
author:  ['korinek_trades']
raw_text:  ['$AAPL has met my long term target \n\nSatisfying A=C and testing the long term TL support level\n\nThe sub waves are corrective and the triangle suggests that this has been a constructive retracement \n\nCircled area holds, 160–&gt;180+ can trade again IMO\n\nVery important spot here https://t.co/vSPR0thPEm']
like_num:  [29]
comment_num:  [1]
retweet_num:  [8]
tweet_activity:  [38]
subjectivity:  [1]
sentiment:  [1]

id:  1610036971723948032
company:  ['apple']
post_date:  2023-01-02T22:15:01Z
author:  ['BStew_ChArTs']
raw_text:  ['$AAPL 🍎\n\nDaily and Weekly charts:\n\nWhat’s your thoughts? Bullish or Bearish?\n\nThere is a lot of support around this current area. \n\nIF AAPL breaks $105 area in 2023 there isn’t much support until around $82\n\nWill be Tech Giant maintain its #1 position? https://t.co/FOyLtSXxop']
like_num:  [40]
comment_num:  [6]
retweet_num:  [6]
tw

# Single field query, query field = text body, query with wrong spelling

Note that spell correction only applied to clean_text

In [20]:
# query on 1 field
query_field = "clean_text" # define the field to search under
query = 'increasy' # define the search term
fields_to_return = "id, company, post_date, author, raw_text, like_num, comment_num, retweet_num, tweet_activity, subjectivity, sentiment" # define the fields to return results
top_n_results = 3 # define the number of top results to return

query_string = "%s: %s" % (query_field, query)

search_results = solr.search(query_string, 
                             **{'fl': fields_to_return, 
                                'rows': top_n_results})

display_results(search_results, query_field)

It seems the query you are searching if not present in the database. Here are some alternatives suggested queries: 
['increase', 'increased', 'increases']


# Multi-fields query

In [21]:
# query on multiple fields
fields_to_return = "id, company, post_date, author, raw_text, like_num, comment_num, retweet_num, tweet_activity, subjectivity, sentiment" # define the fields to return results
top_n_results = 3 # define the number of top results to return

query_string = "company: apple AND clean_text: increase" # AND OR

search_results = solr.search(query_string, 
                             **{'fl': fields_to_return, 
                                'rows': top_n_results})

display_results(search_results, query_field)

id:  1624062449799376896
company:  ['apple']
post_date:  2023-02-10T15:07:16Z
author:  ['StockMKTNewz']
raw_text:  ['Apple $AAPL grew its workforce by 20% between 2020 and 2022\n\nThat compares to a 50% increase at Microsoft, a 60% increase at Google and close to a 100% increase at Facebook, Amazon, and Salesforce - Bloomberg https://t.co/iCJ3slxMvJ']
like_num:  [129]
comment_num:  [5]
retweet_num:  [22]
tweet_activity:  [156]
subjectivity:  [0]
sentiment:  [0]

id:  1626588479244345344
company:  ['apple']
post_date:  2023-02-17T14:24:48Z
author:  ['DividendBreeder']
raw_text:  ['Now that $O and $KO raised their dividend, here’s some more upcoming increase announcements I’m anticipating:\n\n$HD (Feb.)\n$AAPL (April)\n$JNJ (April)\n$PG (April)\n\nAnd maybe, just maybe, AT&amp;T $T will announce a small increase in March.']
like_num:  [75]
comment_num:  [7]
retweet_num:  [4]
tweet_activity:  [86]
subjectivity:  [1]
sentiment:  [1]

id:  1625606452168757248
company:  ['apple']
post_date: 